In [1]:
import pandas as pd
import time
import numpy as np
from scipy import sparse
import warnings
from implicit_markov_chain import ImplicitMarkovChain
from model_helper import save_model
warnings.filterwarnings('ignore')

# Load the data
print("Loading MovieLens 20M data...")
selected_users = pd.read_csv('data/users_selection.csv')
ratings = pd.read_csv('data/ml-20m/ratings.csv')

# print(f"Movies: {movies.shape}")
print(f"Ratings: {ratings.shape}")
print("\nFirst few rows of ratings:")
print(ratings.head())
# print("\nRating distribution:")
# print(ratings['rating'].value_counts().sort_index())

Loading MovieLens 20M data...
Ratings: (20000263, 4)

First few rows of ratings:
   userId  movieId  rating   timestamp
0       1        2     3.5  1112486027
1       1       29     3.5  1112484676
2       1       32     3.5  1112484819
3       1       47     3.5  1112484727
4       1       50     3.5  1112484580


In [ ]:
# np.random.seed(14)

Selected 100 unique users


In [3]:
# Configuration: number of items to leave out for testing per user
leave_k_out = 20  # Change this to test with different k values (e.g., 1, 2, 5, 10)

iterations = 100

for iter in range(iterations):
    print(f"{iter+1}/{iterations}")
    selected_user_ids = np.random.choice(selected_users['x'].unique(), 100, replace=False)
    # print(f"Selected {len(selected_user_ids)} unique users")

    filtered_ratings = ratings[ratings['userId'].isin(selected_user_ids)]
    # print(f"Original ratings shape: {ratings.shape}")
    # print(f"Filtered ratings shape: {filtered_ratings.shape}")

    filtered_ratings_sorted = filtered_ratings.sort_values(['userId', 'timestamp'])

    # Leave Last K Out - generic implementation
    if leave_k_out > 0:
        train_ratings = filtered_ratings_sorted.groupby('userId').apply(lambda x: x.iloc[:-leave_k_out] if len(x) > leave_k_out else pd.DataFrame()).reset_index(drop=True)
        test_ratings = filtered_ratings_sorted.groupby('userId').tail(leave_k_out)
        # Filter out users with insufficient ratings
        user_counts = filtered_ratings_sorted.groupby('userId').size()
        valid_users = user_counts[user_counts > leave_k_out].index
        test_ratings = test_ratings[test_ratings['userId'].isin(valid_users)]
        train_ratings = train_ratings[train_ratings['userId'].isin(valid_users)]
    else:
        # If k=0, use all data for training (no test set)
        train_ratings = filtered_ratings_sorted
        test_ratings = pd.DataFrame(columns=filtered_ratings_sorted.columns)

    print(f"Training set: {train_ratings.shape[0]:,} ratings")
    print(f"Test set: {test_ratings.shape[0]:,} ratings (Leave-{leave_k_out}-out)")

    print("Training implicit feedback Markov chain...")
    model = ImplicitMarkovChain(alpha=2.0)
    start = time.time()
    model.build_transition_matrix_efficient(train_ratings, sample_fraction=1)
    print(f"{time.time() - start:.4f} seconds")

    K = 20
    total_recall = 0
    total_precision = 0
    total_relevant = 0
    num_users_with_test_data = 0
    user_recommendations = {}
    train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])

    for user_id in selected_user_ids:
        user_data = train_ratings_sorted[train_ratings_sorted['userId'] == user_id]
        user_test_ratings = test_ratings[test_ratings['userId'] == user_id]
        
        # Skip users with no test data
        if len(user_test_ratings) == 0:
            continue
        
        num_users_with_test_data += 1
        user_training_samples = len(user_data)
        
        candidate_scores = {}

        # Consider last_k movies (where k = leave_k_out) instead of just the last one
        last_k_movies = user_data['movieId'].tail(leave_k_out).values
        
        # Aggregate probabilities from all last_k movies
        aggregated_probabilities = np.zeros(model.transition_matrix.shape[1])
        
        for movie_id in last_k_movies:
            if movie_id in model.movie_to_idx:
                movie_idx = model.movie_to_idx[movie_id]
                # Sum probabilities from all last_k movies
                aggregated_probabilities += model.transition_matrix[movie_idx]
        
        # Sort indices based on aggregated probabilities
        top_indices = np.argsort(aggregated_probabilities)[-(K*15):][::-1]

        for i in top_indices:
            if len(candidate_scores) == K:
                break
            candidate_movie_id = model.idx_to_movie[i]
            candidate_prob = aggregated_probabilities[i]
            
            if candidate_movie_id not in user_data['movieId'].values:
                # Keep the highest probability for each movie
                if candidate_movie_id not in candidate_scores or candidate_prob > candidate_scores[candidate_movie_id]:
                    candidate_scores[candidate_movie_id] = candidate_prob

        top_movies = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:K]
        candidate_movies = [movie_id for movie_id, score in top_movies]
        
        # validation
        relevant_movies = set(user_test_ratings['movieId'].tolist())
        recommended_movies = set(candidate_movies)
        
        # Calculate hits (intersection of relevant and recommended)
        hits = relevant_movies.intersection(recommended_movies)
        num_hits = len(hits)
        num_relevant = len(relevant_movies)
        
        total_recall += num_hits
        total_precision += num_hits
        total_relevant += num_relevant

    # Calculate metrics
    if total_relevant > 0:
        avg_recall = total_recall / total_relevant
    else:
        avg_recall = 0.0
    
    if num_users_with_test_data > 0:
        avg_precision = total_precision / (num_users_with_test_data * K)
    else:
        avg_precision = 0.0

    print("\n=== Final Results ===")
    print(f"Total users processed: {len(selected_user_ids)}")
    print(f"Users with test data: {num_users_with_test_data}")
    print(f"Leave-{leave_k_out}-out evaluation")
    print(f"Recall@{K}: {avg_recall:.4f}")
    print(f"Precision@{K}: {avg_precision:.4f}")

1/100
Training set: 147,158 ratings
Test set: 2,000 ratings (Leave-20-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 100


100%|██████████| 100/100 [02:50<00:00,  1.70s/it]


200.3622 seconds

=== Final Results ===
Total users processed: 100
Users with test data: 100
Leave-20-out evaluation
Recall@20: 0.0275
Precision@20: 0.0275
2/100
Training set: 150,958 ratings
Test set: 2,000 ratings (Leave-20-out)
Training implicit feedback Markov chain...
Processing user ratings...
User Groups: 100


  6%|▌         | 6/100 [00:07<01:52,  1.19s/it]


KeyboardInterrupt: 

In [4]:
leave_k_out = 100

selected_user_ids = np.random.choice(selected_users['x'].unique(), 100, replace=False)
print(f"Selected {len(selected_user_ids)} unique users")

filtered_ratings = ratings[ratings['userId'].isin(selected_user_ids)]
print(f"Original ratings shape: {ratings.shape}")
print(f"Filtered ratings shape: {filtered_ratings.shape}")

filtered_ratings_sorted = filtered_ratings.sort_values(['userId', 'timestamp'])

train_ratings = filtered_ratings_sorted.groupby('userId').apply(lambda x: x.iloc[:-leave_k_out] if len(x) > leave_k_out else pd.DataFrame()).reset_index(drop=True)
test_ratings = filtered_ratings_sorted.groupby('userId').tail(leave_k_out)

print(f"Training set: {train_ratings.shape[0]:,} ratings")
print(f"Test set: {test_ratings.shape[0]:,} ratings")

print("Training implicit feedback Markov chain...")
model = ImplicitMarkovChain(alpha=2.0)
start = time.time()
model.build_transition_matrix_efficient(train_ratings, sample_fraction=1)
print(f"{time.time() - start:.4f} seconds")

Selected 100 unique users
Original ratings shape: (20000263, 4)
Filtered ratings shape: (133497, 4)
Training set: 123,497 ratings
Test set: 10,000 ratings
Training implicit feedback Markov chain...


100%|██████████| 100/100 [01:52<00:00,  1.12s/it]


126.0156 seconds


In [2]:
from model_helper import load_model
# save_model(model, "models/model_1000_users.pkl")
load_model("models/model_1000_users.pkl")

Model loaded from models/model_1000_users.pkl


In [5]:
# from curses import window

K = 50
total_recall = 0
users_not_considered = 0
recalls = []
total_precision = 0
user_recommendations = {}
train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])

for user_id in selected_user_ids:
    user_data = train_ratings_sorted[train_ratings_sorted['userId'] == user_id]
    user_training_samples = len(user_data)
    
    candidate_scores = {}
    
    window_size = 0
    while len(candidate_scores) < K and window_size < len(user_data):
        # print(window_size)
        window_size += K
        additional_movies = user_data['movieId'].tail(window_size).head(K).tolist()

        
        for movie_id in additional_movies:
            probabilities = model.transition_matrix[model.movie_to_idx[movie_id]]
            top_indices = np.argsort(probabilities)[-(K):][::-1]
            
            movies_added = 0
            for i in top_indices:
                # print(f"movies added: {movies_added}")
                # if movies_added >= K:
                #     break
                    
                candidate_movie_id = model.idx_to_movie[i]
                candidate_prob = probabilities[i]
                
                # Skip if user has already rated this movie
                if candidate_movie_id in user_data['movieId'].values:
                    continue
                    
                # Keep the highest probability for each movie
                if candidate_movie_id not in candidate_scores:
                    candidate_scores[candidate_movie_id] = candidate_prob
                    movies_added += 1
                elif candidate_prob > candidate_scores[candidate_movie_id]:
                    candidate_scores[candidate_movie_id] = candidate_prob
                    
            # if len(candidate_scores) >= K:
            #     break


    top_movies = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)[:K]
    if len(top_movies) < K:
        users_not_considered += 1
        continue
    candidate_movies = [movie_id for movie_id, score in top_movies]
    recommended_movies = set(candidate_movies)
    user_test_ratings = test_ratings[test_ratings['userId'] == user_id]
    relevant_movies = set(user_test_ratings['movieId'].tolist())
    
    # validation
    hits = relevant_movies.intersection(recommended_movies)
    
    user_recall = 0
    if len(relevant_movies) > 0:
        user_recall = len(hits) / len(relevant_movies)
        total_recall += user_recall    
    recalls.append(user_recall)
    print(f"userid: {user_id} - len(top_movies): {len(top_movies)} - training_data: {len(user_data)} - test_data: {len(relevant_movies)} - window: {window_size} - hits: {len(hits)} recall: {user_recall}")

    if len(recommended_movies) > 0:
        user_precision = len(hits) / len(recommended_movies)
        total_precision += user_precision

print(f"total users: {len(selected_user_ids)} - users not consid. {users_not_considered} - total_recall: {total_recall}")
avg_recall = total_recall / (len(selected_user_ids)-users_not_considered)
avg_precision = total_precision / ((len(selected_user_ids)-users_not_considered)*K)

print("\n=== Final Results ===")
print(f"Total users processed: {len(selected_user_ids)}")
print(f"Recall@{K}: {avg_recall:.4f}")
print(f"Precision@{K}: {avg_precision:.4f}")

userid: 63459 - len(top_movies): 50 - training_data: 1330 - test_data: 100 - window: 150 - hits: 8 recall: 0.08
userid: 75210 - len(top_movies): 50 - training_data: 905 - test_data: 100 - window: 200 - hits: 2 recall: 0.02
userid: 39766 - len(top_movies): 50 - training_data: 1394 - test_data: 100 - window: 650 - hits: 5 recall: 0.05
userid: 16676 - len(top_movies): 50 - training_data: 2458 - test_data: 100 - window: 1350 - hits: 4 recall: 0.04
userid: 40666 - len(top_movies): 50 - training_data: 1115 - test_data: 100 - window: 100 - hits: 9 recall: 0.09
userid: 75299 - len(top_movies): 50 - training_data: 2245 - test_data: 100 - window: 2250 - hits: 3 recall: 0.03
userid: 127171 - len(top_movies): 50 - training_data: 993 - test_data: 100 - window: 100 - hits: 4 recall: 0.04
userid: 109186 - len(top_movies): 50 - training_data: 975 - test_data: 100 - window: 50 - hits: 1 recall: 0.01
userid: 10055 - len(top_movies): 50 - training_data: 939 - test_data: 100 - window: 50 - hits: 10 recall

KeyboardInterrupt: 

# Similar Users

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

WEIGH_TIME = False
HALF_LIFE = 365  # Half-life in days for time decay

# Non time-weighted User-Movie Matrix
# train_ratings_sorted = train_ratings.sort_values(['userId', 'timestamp'])
user_movie_matrix = train_ratings.pivot_table(
    index="userId",
    columns="movieId", 
    values="rating",
    aggfunc="sum",
    fill_value=0
)

print(f"User-Movie Matrix shape: {user_movie_matrix.shape}")
print(f"Number of users: {user_movie_matrix.shape[0]}")
print(f"Number of movies: {user_movie_matrix.shape[1]}")

if WEIGH_TIME:
    # Convert timestamp to datetime if it's in Unix format
    if ratings['timestamp'].dtype in ['int64', 'float64']:
        ratings['order_date'] = pd.to_datetime(ratings['timestamp'], unit='s')
    else:
        ratings['order_date'] = pd.to_datetime(ratings['timestamp'])
    
    # Calculate days since most recent rating
    ratings["days_since"] = (ratings['order_date'].max() - ratings['order_date']).dt.days
    
    # Time-decayed weight (more recent ratings have higher weight)
    ratings["time_weight"] = np.exp(-ratings["days_since"] / HALF_LIFE)
    
    # Create weighted ratings matrix
    weighted_ratings = ratings.pivot_table(
        index="userId",
        columns="movieId",
        values="time_weight", 
        aggfunc="sum",
        fill_value=0
    )
    
    # Apply time weighting to the ratings
    user_movie_matrix = np.multiply(weighted_ratings, user_movie_matrix)

# Get the matrix values for similarity computation
user_matrix = user_movie_matrix.values

# Compute cosine similarity among users
print("Computing cosine similarity...")
similarity_matrix = cosine_similarity(user_matrix)

# Convert back to DataFrame with user IDs
user_ids = user_movie_matrix.index
user_similarity_df = pd.DataFrame(similarity_matrix, index=user_ids, columns=user_ids)

print(f"Similarity matrix shape: {user_similarity_df.shape}")

User-Movie Matrix shape: (100, 11089)
Number of users: 100
Number of movies: 11089
Computing cosine similarity...
Similarity matrix shape: (100, 100)


In [7]:
import plotly.graph_objects as go

# Create the heatmap with proper dimensions
fig = go.Figure(
    data=go.Heatmap(
        z=user_similarity_df.values,
        x=user_similarity_df.columns,
        y=user_similarity_df.index,
        colorscale='YlGnBu',
        hoverongaps=False,
        hovertemplate="<b>User 1</b>: %{y}<br><b>User 2</b>: %{x}<br><b>Similarity</b>: %{z:.4f}<extra></extra>"
    )
)

fig.update_layout(
    title=f"User Cosine Similarity Heatmap ({user_similarity_df.shape[0]}×{user_similarity_df.shape[1]})",
    xaxis_title="User ID",
    yaxis_title="User ID",
    width=800,  # Adjust size for better visibility
    height=700,
    xaxis=dict(
        showticklabels=False,  # Hide tick labels for large matrices
        title="User ID (500 users)"
    ),
    yaxis=dict(
        showticklabels=False,  # Hide tick labels for large matrices
        title="User ID (500 users)"
    )
)

print(f"Similarity matrix shape: {user_similarity_df.shape}")
print(f"Expected heatmap dimensions: {user_similarity_df.shape[0]} × {user_similarity_df.shape[1]}")

fig.show()

Similarity matrix shape: (100, 100)
Expected heatmap dimensions: 100 × 100


In [8]:
from sklearn.metrics.pairwise import manhattan_distances
import plotly.graph_objects as go
import numpy as np

# Compute Manhattan distance among users
print("Computing Manhattan distances...")
manhattan_dist_matrix = manhattan_distances(user_movie_matrix.values)

# Convert distance to similarity (distance of 0 = similarity of 1, higher distance = lower similarity)
# Normalize to 0-1 range
max_distance = manhattan_dist_matrix.max()
manhattan_similarity = 1 - (manhattan_dist_matrix / max_distance)

# Convert back to DataFrame
user_ids = user_movie_matrix.index
manhattan_similarity_df = pd.DataFrame(manhattan_similarity, index=user_ids, columns=user_ids)

print(f"Manhattan similarity matrix shape: {manhattan_similarity_df.shape}")
print(f"Value range: [{manhattan_similarity_df.values.min():.4f}, {manhattan_similarity_df.values.max():.4f}]")

# Create the heatmap
fig = go.Figure(
    data=go.Heatmap(
        z=manhattan_similarity_df.values,
        x=manhattan_similarity_df.columns,
        y=manhattan_similarity_df.index,
        colorscale='YlGnBu',
        hoverongaps=False,
        hovertemplate="<b>User 1</b>: %{y}<br><b>User 2</b>: %{x}<br><b>Manhattan Similarity</b>: %{z:.4f}<extra></extra>"
    )
)

fig.update_layout(
    title=f"User Manhattan Similarity Heatmap ({manhattan_similarity_df.shape[0]}×{manhattan_similarity_df.shape[1]})",
    xaxis_title="User ID",
    yaxis_title="User ID",
    width=800,
    height=700,
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False)
)

fig.show()

Computing Manhattan distances...
Manhattan similarity matrix shape: (100, 100)
Value range: [0.0000, 1.0000]


In [18]:
import plotly.express as px

user_id = 1849
# user_id = 72983
distances = user_similarity_df[user_id].copy()
distances = distances.sort_values(ascending=False)
distances = distances.iloc[1:]  # drop self

# Convert to DataFrame for Plotly with user IDs as strings
plot_data = pd.DataFrame({
    'user': distances.index.astype(str),  # Convert to string
    'Cosine similarity': distances.values
})

fig = px.bar(
    plot_data,
    x='user',
    y='Cosine similarity',
    title=f'Sorted Cosine Similarity from User {user_id}',
    labels={'Cosine similarity': 'Similarity Score', 'user': 'Compared User'},
    color='Cosine similarity',
    color_continuous_scale='Bluered',
    hover_data={'user': True, 'Cosine similarity': ':.3f'}
)

fig.update_layout(
    xaxis_title="Users (Sorted by Similarity)",
    yaxis_title="Cosine Similarity",
    hovermode="x unified",
    xaxis={'type': 'category'},  # Force categorical axis
    coloraxis_showscale=False,
    height=600,
    template='plotly_white'
)

fig.update_traces(
    hovertemplate="<b>User %{x}</b><br>Similarity: %{y:.3f}<extra></extra>",
    marker_line_color='black',
    marker_line_width=0.5
)

if len(distances) > 20:
    fig.update_xaxes(rangeslider_visible=True)

fig.show()

In [20]:
from plotly.subplots import make_subplots
from scipy import sparse, stats

def plot_analysis_empirical(customer_name):
    similarities = user_similarity_df[customer_name].copy()
    similarities = similarities.sort_values(ascending=False)
    similarities = similarities.iloc[1:] # drop self
    
    # Convert to DataFrame for Plotly
    plot_data = pd.DataFrame({
        'User': similarities.index.astype(str),
        'Similarity': similarities.values
    })
        
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        row_heights=[0.5,0.5],
        vertical_spacing=0.05,
        subplot_titles=(
            f'<b>{customer_name}</b> - Similarity Analysis',
            'Empirical Distribution'
        )
    )
    
    # Sorted Chi-squared similarities -----------------------------------------------
    fig.add_trace(
        go.Bar(
            x=similarities.index.astype(str),
            y=similarities,
            name='Similarity',
            marker=dict(
                color=similarities,                
                colorscale='Bluered',                  
                colorbar=dict(title='Similarity Scale'),
                showscale=False                        
            ),
            hovertemplate='<b>%{x}</b><br>Similarity: %{y:.3f}<extra></extra>'
        ),
        row=1, col=1
    )
    
    # Empirical distribution modeling -------------------------------
    fig.add_trace(
        go.Histogram(
            x=similarities,
            nbinsx=20,
            # name='Samples',
            marker_color='lightgreen',
            opacity=0.7,
            hovertemplate='<br>Neighbors: %{y:.1f}<extra></extra>'
        ),
        row=2, col=1
    )
    # Mean and Std Dev lines ---------------------------------------------
    mean, std_dev = stats.norm.fit(similarities)
    fig.add_vline(
        x=mean, line_dash="dash", line_color="green",
        annotation_text=f"Mean: {mean:.2f}", 
        annotation_position="bottom right",
        annotation_font=dict(size=12, weight="bold", color="green"),
        row=2, col=1
    )
    fig.add_vline(
        x=mean + std_dev, line_dash="dot", line_color="green",
        annotation_text=f"+1σ: {mean + std_dev:.2f}",
        annotation_font=dict(size=12, weight="bold", color="green"),
        row=2, col=1
    )
    fig.add_vline(
        x=mean - std_dev, line_dash="dot", line_color="green",
        annotation_text=f"-1σ: {mean - std_dev:.2f}",
        annotation_font=dict(size=12, weight="bold", color="green"),
        row=2, col=1
    )
    
    # Fitting a cdf curve for visualization ---------------------------
    similarity_threshold = 0.6
    # Get customer names and their similarities
    most_similar_customers = similarities[similarities.values >= similarity_threshold]
    if len(most_similar_customers) == 0:
        print(f"No similar customers found with threshold {similarity_threshold}")
        return
    # pick the first item
    highest_threshold = min(most_similar_customers.values)
    print(f"Most similar customers (n={len(most_similar_customers)}):")
    
    counts, bin_edges = np.histogram(similarities, bins='auto')
    bin_width = bin_edges[1] - bin_edges[0]
    # Scale factor: Total area under histogram = total counts * bin width
    scale_factor = len(similarities) * bin_width


    # Compute KDE (automatically handles PDF estimation)
    kde = stats.gaussian_kde(similarities)
    
    # Generate points for plotting
    x_pdf = np.linspace(similarities.min(), 
                        similarities.max(), 
                        100)
    y_pdf = kde(x_pdf)

    
    # Generate normal curve points
    fig.add_trace(go.Scatter(
        # x=res.cdf.quantiles, y=res.cdf.probabilities* scale_factor, mode='lines',line=dict(color='teal', width=2),
        x=x_pdf, y=y_pdf*scale_factor, mode='lines',line=dict(color='teal', width=2),
        hoverinfo='text+x'),
        row=2, col=1    
    )
    
    # 97.5th percentile line -------------------------------------------------------
    fig.add_vline(
        x=highest_threshold, line_dash="dash", line_color="blue",
        annotation_text=f"({len(most_similar_customers)}): {highest_threshold:.2f}",
        annotation_position="bottom right",
        annotation_font=dict(size=12, color="blue"),
        row=2, col=1    
    )
    
    # 97.5th pct. customers --------------------------------------------------------
    fig.add_trace(go.Scatter(
        x=similarities,
        y=np.zeros(len(most_similar_customers)),
        mode='markers',
        marker=dict(color='blue', size=10),
        text=most_similar_customers.index,
        textposition='top center',
        hoverinfo='text+x',
        hovertext=[f"<b>{cust}</b><br>Distance: {dist:.3f}" 
                   for cust, dist in zip(most_similar_customers.index, similarities)],
        name='Lowest 10%'),
        row=2, col=1
    )
    
    # Customize layout
    fig.update_layout(
        height=1200,
        hovermode="x unified",
        showlegend=False,
        yaxis1={'title': 'Similarity index'},
        yaxis2={'title': 'Top similar customers'},
        template='plotly_white'
    )
    
    fig.show()

# plot_analysis_empirical(8805)
# plot_analysis_empirical(72983)
plot_analysis_empirical(3743)

# user_id = 72983, 6630


Most similar customers (n=1):


In [26]:
from sklearn.preprocessing import MinMaxScaler

def normalize_series(s):
    scaler = MinMaxScaler()
    return pd.Series(
        scaler.fit_transform(s.values.reshape(-1, 1)).flatten(),
        index=s.index
    )

def get_similar_customers(customer_i, top_k = 10, similarity_threshold = 0.6):
    similarities = user_similarity_df[customer_i].copy()
    similarities = similarities.sort_values(ascending=False)
    similarities = similarities.iloc[1:] # drop self
    # Get customer names and their similarities
    most_similar_customers = similarities[similarities.values >= similarity_threshold]
    if len(most_similar_customers) == 0:
        print(f"No similar customers found with threshold {similarity_threshold}")
        return
    print(f"Most similar customers (n={len(most_similar_customers)}):")
    return most_similar_customers.nlargest(top_k)


def get_recommendation_based_on_similarity(customer_i, top_k=20):
    similar_customers = get_similar_customers(customer_i)
    similar_customers_name = similar_customers.index
    similar_customers_similarity = similar_customers.values
    
    print(f"\033[1mTop similar customers to {customer_i}:\033[0m")
    print(f"Customers  -  Similarity")
    for name, similarity in zip(similar_customers_name, similar_customers_similarity):
        print(f"{name}:  \t{similarity:.4f}") 
    print("")
    
    # Get movies that customer_i has already rated from the user_movie_matrix
    customer_rated_movies = user_movie_matrix.loc[customer_i]
    customer_rated_movies = customer_rated_movies[customer_rated_movies > 0].index.tolist()
    
    # Dictionary to store movie scores
    movie_scores = {}
    movie_weights = {}
    
    # For each similar customer
    for similar_customer, similarity_score in zip(similar_customers_name, similar_customers_similarity):
        # Skip if similar customer is not in the matrix
        if similar_customer not in user_movie_matrix.index:
            continue
            
        # Get movies rated by similar customer from the matrix
        similar_customer_ratings = user_movie_matrix.loc[similar_customer]
        rated_movies = similar_customer_ratings[similar_customer_ratings > 0]
        
        for movie_id, rating in rated_movies.items():
            # Skip movies that customer_i has already rated
            if movie_id in customer_rated_movies:
                continue
                
            # Calculate weighted score (rating * similarity)
            weighted_score = rating * similarity_score
            
            # Aggregate scores
            if movie_id in movie_scores:
                movie_scores[movie_id] += weighted_score
                movie_weights[movie_id] += similarity_score
            else:
                movie_scores[movie_id] = weighted_score
                movie_weights[movie_id] = similarity_score
    
    # Calculate final scores (average weighted rating)
    final_scores = {}
    for movie_id, total_score in movie_scores.items():
        total_weight = movie_weights[movie_id]
        if total_weight > 0:  # Avoid division by zero
            final_scores[movie_id] = total_score / total_weight
    
    # Convert to probability distribution using softmax
    if final_scores:
        # Get top movies by raw score first
        top_movies_raw = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)[:top_k*3]  # Get more candidates
        
        # Extract scores for softmax
        movie_ids = [movie_id for movie_id, score in top_movies_raw]
        scores = np.array([score for movie_id, score in top_movies_raw])
        
        # Apply softmax to convert to probabilities
        exp_scores = np.exp(scores - np.max(scores))  # Subtract max for numerical stability
        probabilities = exp_scores / np.sum(exp_scores)
        
        # Create probability distribution dictionary
        prob_distribution = {movie_id: prob for movie_id, prob in zip(movie_ids, probabilities)}
        
        # Get top K by probability (should be same order as by score due to softmax monotonicity)
        top_recommendations = list(prob_distribution.items())[:top_k]
        
        # Print recommendations
        print(f"\033[1mTop {top_k} Recommendations for {customer_i} (Probability Distribution):\033[0m")
        print("Movie ID  -  Probability")
        total_prob = 0
        for movie_id, prob in top_recommendations:
            print(f"{movie_id}:  \t{prob:.4f}")
            total_prob += prob
        print(f"Total probability for top {top_k}: {total_prob:.4f}")
        
        return prob_distribution
    else:
        print("No recommendations found.")
        return {}


def recommend_parts_for_customer(customer_i, top_k):
    # Markov Chain boost
    last_ordered_part = df[df["Customer"] == customer_i].sort_values("Order_Ship_Date")["Part_Number"].iloc[-1]
    markov_suggestions = transition_df.loc[last_ordered_part].sort_values(ascending=False)
    print(f"\033[1mMarkov Chain suggestions for {customer_i}:\033[0m")
    print(f"{markov_suggestions.head(top_k)}")
    print("")
    
    # Find similar customers
    similar_customers = get_similar_customers(customer_i, top_k)
    similar_customers_name = similar_customers.index
    similar_customers_similarity = similar_customers.values
    print(f"\033[1mTop similar customers to {customer_i}:\033[0m")
    print(f"Customers  -  Similarity")
    for name, similarity in zip(similar_customers_name, similar_customers_similarity):
        print(f"{name}:  \t{similarity:.4f}") 
    print("")
    
    # Aggregate parts they have bought
    similar_movies = user_movie_matrix.loc[similar_customers_name].sum()
    # Returns Partx x 1 matrix
    print(f"Similar purchased parts: {len(similar_movies)}")
    print(f"Similar purchased parts: {len(similar_movies)}\n{similar_movies.sort_values(ascending=False)[1:10]}")
    
    # Remove parts already bought by customer_i
    already_rated = user_movie_matrix.loc[customer_i]
    parts_to_recommend = similar_movies[already_rated == 0]
    print(f"Number of parts to recommend: {len(parts_to_recommend)}")

    norm_collab = normalize_series(parts_to_recommend)
    print(f"Normalized collab: \n{norm_collab.sort_values(ascending=False).head(top_k)}")
    norm_markov = normalize_series(markov_suggestions)
    print(f"Normalized markov: \n{norm_markov.sort_values(ascending=False).head(top_k)}")
    # Union of all unique part numbers across both sources
    all_parts = norm_collab.index.union(norm_markov.index)
    
    norm_collab = norm_collab.reindex(all_parts, fill_value=0)
    norm_markov = norm_markov.reindex(all_parts, fill_value=0)
    
    top_collab_items = norm_collab.sort_values(ascending=False).head(top_k).index
    print(f"Top {top_k} collaborative items with Markov values:")
    for item in top_collab_items:
        collab_val = norm_collab[item]
        markov_val = norm_markov.get(item, 0)
        print(f"{item}: Collab.={collab_val:.4f},\t Markov={markov_val:.4f}")
    
    top_markov_items = norm_markov.sort_values(ascending=False).head(top_k).index
    print(f"Top {top_k} Markov items with Collab values:")
    for item in top_markov_items:
        markov_val = norm_markov[item]
        collab_val = norm_collab.get(item, 0)
        print(f"{item}: Collab.={collab_val:.4f},\t Markov={markov_val:.4f}")

    # Combine with weights
    COLLAB = 1
    MARKOV = 0.0
    final_scores = COLLAB * norm_collab + MARKOV * norm_markov
    top_final_scores_items = final_scores.sort_values(ascending=False).head(top_k).index
    print(f"Top {top_k} collaborative items with Final score values:")
    for item in top_collab_items:
        markov_val = norm_markov.get(item,0)
        collab_val = norm_collab[item]
        score = final_scores[item]
        print(f"{item}: Score={score:.4f},\t Collab.={collab_val:.4f},\t Markov={markov_val:.4f}")
    print(f"Final: \n{final_scores.sort_values(ascending=False).head(top_k)}")
    for item in top_markov_items:
        markov_val = norm_markov[item]
        collab_val = norm_collab.get(item,0)
        score = final_scores[item]
        print(f"{item}: Score={score:.4f},\t Collab.={collab_val:.4f},\t Markov={markov_val:.4f}")
    print(f"\033[1mRecommendations based on similarity:\033[0m")
    print(f"{final_scores.sort_values(ascending=False).head(top_k)}")
    
    return final_scores.sort_values(ascending=False).head(top_k)

# recommendations = recommend_parts_for_customer("Cust110", top_k=10)
# print("Recommended parts:", recommendations.index.tolist())
# transition_df.loc[recommendations.index.tolist()[0]].sort_values(ascending=False).head()
# len(get_similar_customers(6630))
get_recommendation_based_on_similarity(3743)

Most similar customers (n=1):
Top similar customers to 3743:
Customers  -  Similarity
49024:  	0.6236

Top 20 Recommendations for 3743 (Probability Distribution):
Movie ID  -  Probability
350:  	0.0267
750:  	0.0267
912:  	0.0267
913:  	0.0267
1214:  	0.0267
1218:  	0.0267
1228:  	0.0267
1261:  	0.0267
1263:  	0.0267
1267:  	0.0267
1297:  	0.0267
1674:  	0.0267
1732:  	0.0267
1997:  	0.0267
2138:  	0.0267
2401:  	0.0267
2528:  	0.0267
2795:  	0.0267
3181:  	0.0267
3498:  	0.0267
Total probability for top 20: 0.5336


{350: np.float64(0.02667925137575028),
 750: np.float64(0.02667925137575028),
 912: np.float64(0.02667925137575028),
 913: np.float64(0.02667925137575028),
 1214: np.float64(0.02667925137575028),
 1218: np.float64(0.02667925137575028),
 1228: np.float64(0.02667925137575028),
 1261: np.float64(0.02667925137575028),
 1263: np.float64(0.02667925137575028),
 1267: np.float64(0.02667925137575028),
 1297: np.float64(0.02667925137575028),
 1674: np.float64(0.02667925137575028),
 1732: np.float64(0.02667925137575028),
 1997: np.float64(0.02667925137575028),
 2138: np.float64(0.02667925137575028),
 2401: np.float64(0.02667925137575028),
 2528: np.float64(0.02667925137575028),
 2795: np.float64(0.02667925137575028),
 3181: np.float64(0.02667925137575028),
 3498: np.float64(0.02667925137575028),
 3505: np.float64(0.02667925137575028),
 3761: np.float64(0.02667925137575028),
 5296: np.float64(0.02667925137575028),
 5299: np.float64(0.02667925137575028),
 5989: np.float64(0.016181783937573),
 21: n